In [ ]:
import os
import json
import random
import numpy as np 
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import plot_utils as plu


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})

# Load data as np.array saved in a dictionnary
also: save settings, compute counts and fixed sites boolean array 

In [ ]:
print()
print("#################### INIT ####################")

In [ ]:
if not os.path.exists(outDir):
    os.makedirs(outDir)
    print('- Creating {dir}'.format(dir=outDir))

In [ ]:
print('- color panel is', colpal)
sns.set_palette(colpal.values())
print('- infile is', infiles.keys())
sns.palplot(sns.color_palette())

In [ ]:
# set the seed so that the same real individual are subsampled (when needed) 
# to ensure consistency of the scores when adding a new model or a new sumstat
np.random.seed(3)
random.seed(3)

# Load  data
print("- Loading data...")
datasets, keptsnpdic, sample_info = dict(), dict(), dict()
for k,fname in infiles.items():
        dattmp = pd.read_csv(fname, sep=' ', header=None, compression='infer')
        if dattmp.shape[0]> nsub:
            dattmp = dattmp.drop(
                index = np.sort(np.random.choice(np.arange(dattmp.shape[0]), size=dattmp.shape[0]-nsub, replace=False))
            )
        # overwrite file first column to set the label name chosen in infiles (eg GAN, etc):
        dattmp[0] = k 
        sample_info[k] =  pd.DataFrame({'label':dattmp[0], 'ind':dattmp[1]})
        datasets[k] = np.array(dattmp.loc[:,2:])
        
        # transformations can be maf filtering, recoding into major=0/minor=1 format
        if not transformations is None:
            datasets[k], keptsnpdic[k] = plu.datatransform(datasets[k],**transformations)
        print('    -', k, dattmp.shape, "loaded from", fname)
        
sampleinfo = pd.DataFrame(np.concatenate(list(sample_info.values())), columns=['label','id'])        
print("- Dictionnary of datasets:",len(datasets))
#del dattmp        

In [ ]:
# Save settings for future ref
json.dump(dict({'infiles':infiles, 
                'colpal':colpal, 
                'position_fname':position_fname,
                'settings':dict({'outDir':outDir, 'nsub':nsub}),
                'script':'new'
               }),
          open(outDir + 'settings.json', 'w')
         )

In [ ]:
# Compute counts of "1" allele (could be derived, alternative or minor allele depending on the encoding)
# And check whether some sites are fixed
# matching_SNPs will be set to True if all datasets have the same nb of SNPs
# in this case we automatically consider that there can be a one-to-one comparison
# ie 1st SNP of generated datset should mimic the 1st real SNP and so on
categ = infiles.keys()
ac_d, ac_scaled = dict(), dict()
nindiv = dict()
is_fixed_dic = dict()

print('- Check whether some sites are fixed')
for cat, d in datasets.items():
    nindiv[cat] = d.shape[0]
    ac_d[cat] = np.sum(d, axis=0)
    ac_scaled[cat]  = ac_d[cat]/nindiv[cat]
    is_fixed_dic[cat] = (ac_d[cat]%nindiv[cat]==0)
    print(f"    - {is_fixed_dic[cat].sum()} fixed sites in {cat}")

# is site fixed in at least one of the dataset ?
# requires to have the same number of SNPs for all datasets 
# (makes sense for "matching" SNPs)
print('- Is site fixed in at least one of the dataset')
if plu.all_same([d.shape[1] for d in datasets.values()]):
    matching_SNPs = True
    is_fixed = np.vstack([is_fixed_dic[cat] for cat in categ]).any(axis=0)
    print(f"    - {is_fixed.sum()} sites fixed in at least one dataset")
    [print("    - {count} fixed SNPs in {cat} that are not fixed in Real".format(
        count= ((is_fixed_dic[cat]) & (~is_fixed_dic['Real'])).sum(),
        cat=cat
    )) for cat in categ]
else:    
    matching_SNPs = False
    is_fixed = None
print(f'- Matching SNPs?: {matching_SNPs}')

In [ ]:
print("#################### INIT DONE ####################")